In [1]:
from ultralytics import YOLO
import cv2
import ast


import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using CPU. Note: This module is much faster with a GPU.


In [2]:
def draw_boxes(frame, boxes):
    x1, y1, x2, y2= boxes  # Get coordinates of the bounding box
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 10)

In [3]:
def draw_plate(frame, box):
    x1, y1, x2, y2= box  # Get coordinates of the bounding box
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 10)

In [4]:
  import PIL.Image
  if not hasattr(PIL.Image, 'Resampling'):  # Pillow<9.0
    PIL.Image.Resampling = PIL.Image
  # Now PIL.Image.Resampling.BICUBIC is always recognized.

In [5]:
results = {}

mot_tracker = Sort()

# load models
coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO('/home/harshit/Desktop/project/license_plate_detector.pt')

# load video
cap = cv2.VideoCapture('/home/harshit/Desktop/project/sample.mp4')

In [6]:
cv2.destroyAllWindows()

In [8]:
vehicles = [2, 3, 5, 7]

# read frames
frame_nmr = -1
ret = True
import time
while ret:
    start_time = time.time()
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection

            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

            # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))
        # draw_boxes(frame,detections_)
            # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate
            # draw_plate(frame,[int(x1),int(y1),int(x2),int(y2),0])
                # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                    # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                    # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                    # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)
                print(license_plate_text)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
                    car_bbox = results[frame_nmr][car_id]['car']['bbox']
                    license_plate_bbox = results[frame_nmr][car_id]['license_plate']['bbox']
                    draw_boxes(frame,car_bbox)
                    draw_plate(frame,license_plate_bbox)

                    H, W, _ = license_plate_crop.shape
                    # import ast

                    # Assuming results is a dictionary
                    # Assuming results is a dictionary
                    car_bbox = results[frame_nmr][car_id]['car']['bbox']
                    car_x1, car_y1, car_x2, car_y2 = car_bbox  # Unpack the list directly


                    # Now you have car_x1, car_y1, car_x2, car_y2 containing the bbox coordinates
                    # Assuming results is a dictionary
                    license_plate_bbox = results[frame_nmr][car_id]['license_plate']['bbox']
                    x1, y1, x2, y2 = license_plate_bbox  # Unpack the list directly


                    try:
                        frame[int(car_y1) - H - 100:int(car_y1) - 100,
                            int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = license_plate_crop

                        frame[int(car_y1) - H - 400:int(car_y1) - H - 100,
                            int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = (255, 255, 255)

                      # Calculate the text width and height
                        (text_width, text_height), _ = cv2.getTextSize(
                                                        license_plate_text,
                                                        cv2.FONT_HERSHEY_SIMPLEX,
                                                        4.3,
                                                        17)


                        # Assuming license_plate_bbox is a tuple or list containing (x1, y1, x2, y2) coordinates
                        license_plate_bbox = results[frame_nmr][car_id]['license_plate']['bbox']
                        x1, y1, x2, y2 = license_plate_bbox

                        # Get the license plate number from the DataFrame
                        license_plate_number = license_plate_text

                        # Calculate the position to put the text
                        text_width, text_height = cv2.getTextSize(license_plate_number, cv2.FONT_HERSHEY_SIMPLEX, 4.3, 17)[0]
                        text_position = ((x2 + x1 - text_width) // 2, (y1 - 250 + text_height) // 2)

                        # Put the text on the frame
                        cv2.putText(frame,
                                license_plate_text,
                                (int((car_x2 + car_x1 - text_width) / 2), int(car_y1 - H - 250 + (text_height / 2))),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                4.3,
                                (0, 0, 0),
                                17)
                    except:
                        pass

                



                    width = 800  # Desired width
                    height = 600  # Desired height
                    resized_image = cv2.resize(frame, (width, height))
                    # Display the resized image with a specific window size
                    cv2.namedWindow('Resized Image', cv2.WINDOW_NORMAL)  # Create a resizable window
                    cv2.imshow('Resized Image', resized_image)
                            # Check for 'q' key to exit
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                     break
  
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds") 
cv2.destroyAllWindows()    

 
                    


0: 384x640 22 cars, 1 bus, 3 trucks, 8.9ms
Speed: 21.6ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 5.8ms
Speed: 2.0ms preprocess, 5.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


AP05JEO
None

0: 384x640 23 cars, 1 bus, 3 trucks, 5.9ms
Speed: 1.9ms preprocess, 5.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 5.7ms
Speed: 1.4ms preprocess, 5.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
AP05JEO
None

0: 384x640 23 cars, 1 bus, 2 trucks, 7.3ms
Speed: 2.2ms preprocess, 7.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 5.7ms
Speed: 1.7ms preprocess, 5.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
AP05JEO
None

0: 384x640 23 cars, 2 trucks, 7.6ms
Speed: 2.1ms preprocess, 7.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 5.7ms
Speed: 1.7ms preprocess, 5.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
AP05JEO
None

0: 384x640 23 cars, 2 trucks, 6.1ms
Speed: 2.4ms preprocess, 6.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 0s, 5.8ms
Speed: 1.9ms prepr

KeyboardInterrupt: 

In [ ]:
frame_nmr


In [ ]:
write_csv(results,'/home/harshit/Desktop/project/test.csv')